In [4]:
import pandas as pd
from urllib.parse import urlparse

In [99]:
df=pd.read_csv("C:\My Files\Project_data_2\shopify_orders.csv")
df1=pd.read_excel("C:\My Files\Project_data_2\Fashion_Ecommerce.xlsx")

In [ ]:
#Checking for duplicates
df.duplicated().sum()

#Converting created date
df['s_created_at']=pd.to_datetime(df['s_created_at'])

In [ ]:
data = df[['s_created_at','s_fulfillment_status','s_subtotal_price']]
data['Month']=data['s_created_at'].dt.month
data['Year']=data['s_created_at'].dt.year


Total sales for year 2022 and 2023

In [103]:
years=[2022,2023]
for i in years:
    sales=df1[df1["Year"]==i]["s_subtotal_price"].sum()
    print(f"Total sales in {i} is {sales :.2f}")

Total sales in 2022 is 56544250.68
Total sales in 2023 is 60996421.70


YoY growth for 2022 and 2023

In [25]:
#Calculating YoY
year_sales=[]
year=[2019,2020,2021,2022,2023]
for j in year:
    sales=data[(data['Year'] == j) & (data['s_fulfillment_status'] == "fulfilled")]['s_subtotal_price'].sum()
    year_sales.append(sales)
yoy_sales=[]
for i in range(len(year_sales)-1):
    yoy=((year_sales[i+1]-year_sales[i])/year_sales[i])*100
    yoy_sales.append(yoy)
print(f"Growth in the sales in the year 2022 is {yoy_sales[2] :.2f} %")
print(f"Growth in the sales in the year 2023 is {yoy_sales[3] :.2f} %")

Growth in the sales in the year 2022 is 14.87 %
Growth in the sales in the year 2023 is 10.82 %


There is a decline in the growth in sales

Calculation of Metrices

In [46]:
# % Increase in Order Count
years=[2021,2022,2023]
orders=[]
percent_increase=[]
for i in years:
    orders.append(df1[df1["Year"]==i]["order_seq"].size)
for i in range(len(orders)-1):
    percent_increase.append(((orders[i+1]-orders[i])/orders[i])*100)
    print(f"% increase in orders for {years[i+1]} is {percent_increase[i] :.2f} %")
    
print()

# % Decrease in Cancel Order Count
orders_cancel=[]
percent_increase_cancel=[]
for i in years:
    orders_cancel.append(df1[(df1["Year"]==i) & (df1["s_cancel_reason"].notna())]["order_seq"].size)
for i in range(len(orders)-1):
    percent_increase_cancel.append(((orders_cancel[i+1]-orders_cancel[i])/orders_cancel[i])*100)
    print(f"% Decrease in cancelled orders for {years[i+1]} is {percent_increase_cancel[i] :.2f} %")

% increase in orders for 2022 is 11.12 %
% increase in orders for 2023 is 3.40 %

% Decrease in cancelled orders for 2022 is -7.49 %
% Decrease in cancelled orders for 2023 is -12.24 %


%Customer Retention Calculation for 2022 and 2023

In [26]:
customers21=df1[df1['Year']== 2021]['s_customer_id'].unique()
customers22=df1[df1['Year']== 2022]['s_customer_id'].unique()
customers23=df1[df1['Year'] == 2023]['s_customer_id'].unique()

retained_cust22=0
for i in customers21:
    for j in range(len(customers22)):
        if i == customers22[j]:
            retained_cust22+=1

retained_cust23=0
for i in customers22:
    for j in range(len(customers23)):
        if i == customers23[j]:
            retained_cust23+=1
cust_ret22=(retained_cust22/len(customers21))*100
cust_ret23=(retained_cust23/len(customers22))*100

print(f"%Customer Retention for 2022 is {cust_ret22 :.2f}%")
print(f"%Customer Retention for 2023 is {cust_ret23 :.2f}%")

%Customer Retention for 2022 is 38.67%
%Customer Retention for 2023 is 37.77%
6429
6882


Cancelled Orders Analysis

In [64]:
#Creation of new dataframe having only cancelled orders
data2 = df1[df1['s_cancel_reason'].notna()]

In [68]:
#Top 5 places with maximum cancelled orders for the years 2021, 2022 and 2023
data2[data2['Year']==2022]['Place'].value_counts().head(5)

Place
Maharashtra    224
Tamil Nadu     188
Karnataka      170
Kerala         162
Telangana      121
Name: count, dtype: int64

In [69]:
data2[data2['Year']==2023]['Place'].value_counts().head(5)

Place
Tamil Nadu     246
Maharashtra    136
Karnataka      134
Telangana      121
Kerala          93
Name: count, dtype: int64

In [70]:
#Top 5 payment methods with maximum cancelled orders for the years 2021, 2022 and 2023
data2[data2['Year']==2023]['s_gateway'].value_counts().head(5)

s_gateway
Cash on Delivery (COD)    1113
Pay through GPay/paytm     102
Paypal                      69
Razorpay Secure             60
Bank Deposit                19
Name: count, dtype: int64

In [71]:
data2[data2['Year']==2022]['s_gateway'].value_counts().head(5)

s_gateway
Cash on Delivery (COD)                    938
Pay through GPay/paytm                    430
Bank Deposit                               96
Paypal                                     55
razorpay_cards_upi_netbanking_wallets_     27
Name: count, dtype: int64

In [73]:
data2[data2['Year']==2021]['s_gateway'].value_counts().head(5)

s_gateway
Cash on Delivery (COD)    989
Pay through GPay/paytm    313
Bank Deposit              243
Paypal                    109
payu_india                 33
Name: count, dtype: int64

Vendor Analysis

In [ ]:
vendor_sales22=df1[df1["Year"]==2022].groupby(["Year","s_vendor"])["s_subtotal_price"].sum().sort_values(ascending=False).reset_index().tail(10)
vendor_sales22

In [ ]:
vendor_sales23=df1[df1["Year"]==2023].groupby(["Year","s_vendor"])["s_subtotal_price"].sum().sort_values(ascending=False).reset_index().tail(10)
vendor_sales23

In [19]:
avg_cancel_orders22=df1[(df1["Year"]==2023) & (df1["s_cancel_reason"].notna())].groupby(["s_vendor"])["order_seq"].size().sort_values(ascending=False).reset_index()

Advertisement Analysis 

In [29]:
df5=pd.read_excel("C:\My Files\Project_data_2\Ad_source.xlsx")

In [39]:
#Combining lv and fv source for reduction in null values
df5['Source'] = df5['s_fv_utm_source'].fillna(df5['s_lv_utm_source'])

In [43]:
#Checking for percentage of null values
df5['Source'].isnull().sum()/df5['Source'].shape[0]

#It has reduced which is good

0.1395620295952239

In [68]:
#Extracting the name of the social media app base on the mapping given
source_map = {
    'FBORG': 'Facebook',
    'INSSTY': 'Instagram',
    'NLEMAIL': 'Mail',
    'WAMSG': 'Whatsapp',
    'pinterest': 'Pinterest',
    'YTRLS': 'YouTube',
    'direct': 'Company Website'
}

# Convert to string and apply keyword matching
df5['Social_app'] = [
    next((v for k, v in source_map.items() if k in str(text)), '')
    for text in df5['Source']
]


In [69]:
df5['year']=df5['s_created_at'].dt.year
df5['year'] = df5['year'].astype('Int64')

In [86]:
df5['Social_app'].value_counts()

Social_app
             60593
Whatsapp      9550
Mail          7952
Facebook      2730
Instagram     1403
YouTube         15
Name: count, dtype: int64

In [72]:
year = [2022, 2023]
for i in year:
    total = df5[(df5["Social_app"] != "") & (df5['year'] == i)]["s_subtotal_price"].sum()
    orders = df5[(df5["Social_app"] != "") & (df5['year'] == i)]["order_seq"].unique().shape[0]
    customers = df5[(df5["Social_app"] != "") & (df5['year'] == i)]["s_customer_id"].unique().shape[0]
    print(i)
    print(f"Orders received is {orders}")
    print(f"Customers is {customers}")
    print(f"Sales contributed by app in is {total :.2f}")

2022
Orders received is 5314
Customers is 520
Sales contributed by app in is 17267298.75
2023
Orders received is 6412
Customers is 394
Sales contributed by app in is 4838851.10


In [76]:
year = [2022, 2023]
for i in year:
    total = (df5[(df5["Social_app"] != "") & (df5['year'] == i)]["s_subtotal_price"].sum() / df5[df5['year']==i]["s_subtotal_price"].sum())*100
    orders = (df5[(df5["Social_app"] != "") & (df5['year'] == i)]["order_seq"].unique().shape[0] / df5[df5['year']==i]["order_seq"].unique().shape[0])*100
    customers = (df5[(df5["Social_app"] != "") & (df5['year'] == i)]["s_customer_id"].unique().shape[0] / df5[df5['year']==i]["order_seq"].unique().shape[0])*100
    print(i)
    print(f"%Orders received through the apps is {orders :.2f}")
    print(f"%Customers who arrived at the website through apps is {customers :.2f}")
    print(f"Sales contributed by the apps in is {total :.2f}")

2022
%Orders received through the apps is 30.89
%Customers who arrived at the website through apps is 3.02
Sales contributed by the apps in is 31.50
2023
%Orders received through the apps is 36.70
%Customers who arrived at the website through apps is 2.25
Sales contributed by the apps in is 40.91


In [97]:
#Social Media
for i in year:
    sales_contri = (
    df5[(df5["Social_app"] != "") & (df5['year'] == i)]
    .groupby("Social_app")["s_subtotal_price"]
    .sum() * 100 /
    df5[(df5["Social_app"] != "") & (df5['year'] == i)]["s_subtotal_price"].sum()
)

    
    print(f"\nYear {i}")
    print("% Sales contribution by each app:")
    print(sales_contri.round(2))


Year 2022
% Sales contribution by each app:
Social_app
Facebook      9.56
Instagram     7.21
Mail         49.48
Whatsapp     33.68
YouTube       0.06
Name: s_subtotal_price, dtype: float64

Year 2023
% Sales contribution by each app:
Social_app
Facebook      9.51
Instagram    11.00
Mail         49.90
Whatsapp     29.55
YouTube       0.04
Name: s_subtotal_price, dtype: float64
